[<img src="../header.svg">](../index.ipynd)

---

# Simulation of Drug Transport across a Virtual Skin Membrane

### Initialize UG4 (for 2D and standard algebra)

In [ ]:
InitUG(2, AlgebraType("CPU", 1));
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

### Create Domain

In [ ]:
requiredSubsets = {"LIP", "COR", "BOTTOM_SC", "TOP_SC"}
gridName = "skin2d-aniso.ugx"
numRefs = 2

In [ ]:
dom = util.CreateDomain(gridName, numRefs, requiredSubsets)

### Create Approximation space

In [ ]:
approxSpaceDesc = { fct = "c", type = "Lagrange", order = 1 }

In [ ]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct(approxSpaceDesc.fct, approxSpaceDesc.type, approxSpaceDesc.order)
approxSpace:init_levels()
approxSpace:init_top_surface()
print("Approximation space:")
approxSpace:print_statistic()

## Create a convection-diffusion-equation
Model parameter

In [ ]:
K={
    ["LIP"] = 1.0, ["COR"] = 1.0,
}

D={
     ["LIP"] = 1, ["COR"] = 0.1, 
}


In [ ]:
elemDisc ={}

elemDisc["COR"] = ConvectionDiffusion("c", "COR", "fv1")
elemDisc["COR"]:set_diffusion(K["COR"]*D["COR"])
elemDisc["COR"]:set_mass_scale(K["COR"])

elemDisc["LIP"] = ConvectionDiffusion("c", "LIP", "fv1")
elemDisc["LIP"]:set_diffusion(K["LIP"]*D["LIP"])
elemDisc["LIP"]:set_mass_scale(K["LIP"])

In [ ]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(1.0, "c", "TOP_SC")
dirichletBnd:add(0.0, "c", "BOTTOM_SC")

In [ ]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc["LIP"])
domainDisc:add(elemDisc["COR"])
domainDisc:add(dirichletBnd)

## Create  solver

In [ ]:
lsolver=SuperLU()

## Solve transient problem

In [ ]:
u = GridFunction(approxSpace)
u:set(0.0)

In [ ]:
startTime = 0.0
endTime = 1000.0
dt=25.0

In [1]:
util.SolveLinearTimeProblem(u, domainDisc, lsolver, VTKOutput(), "vtk/skin",
                            "ImplEuler", 1, startTime, endTime, dt); 

LUA-ERROR: ---

* Initializing: paths... done, bridge... done, plugins... done                 *
LUA-ERROR! Call stack:
   1  buffer:1        
LUA-ERROR: 
[string "buffer"]:1: attempt to index global 'util' (a nil value)
 % ABORTING script parsing.


In [14]:
timeDisc=ThetaTimeStep(domainDisc, 1.0)

timeInt = LinearTimeIntegrator(timeDisc)
timeInt:set_linear_solver(lsolver)
timeInt:set_time_step(0.25)


In [ ]:
timeInt:apply(u, endTime, u, startTime)

In [ ]:
timeInt = ConstStepLinearTimeIntegrator(timeDisc)
timeInt:set_linear_solver(lsolver)
timeInt:set_time_step(dt)

In [ ]:
timeInt:apply(u, endTime, u, startTime)